In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.9 MB/s eta 0:00:00


In [ ]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/distilbert-base-nli-stsb-mean-tokens"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [ ]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [ ]:
import pickle
embeddings_dataset = pickle.load(open('Distillbert_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 69
})

In [ ]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
question = ["Therapy for ADHD", "Not able to focus","Child not sleeping properly","bed wetting"]
question_embedding = get_embeddings([question[-2]]).cpu().detach().numpy()
#question_embedding

In [ ]:
scores, samples = embeddings_dataset.get_nearest_examples(
    "Embeddings", question_embedding, k=5
)

In [ ]:
import pandas as pd

samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=False, inplace=True)

In [ ]:
for _, row in samples_df.iterrows():
    print(f"TITLE: {row.Title}")
    print(f"SCORE: {row.scores}")
    print(f"URL: {row.URL}")
    print("=" * 50)
    print()

TITLE: Early Signs of Autism in Babies
SCORE: 304.6646728515625
URL: https://www.parents.com/baby/health/autism/early-signs-of-autism-in-babies

TITLE: Childhood Fears: What Kids Are Scared of -- and Why!
SCORE: 278.68292236328125
URL: https://www.parents.com/baby/development/intellectual/what-kids-are-scared-of-and-why

TITLE: Separation and Social Anxiety in Kids
SCORE: 276.41400146484375
URL: https://www.parents.com/toddlers-preschoolers/how-to-help-your-kids-adjust-when-they-go-back-to-daycare-and-school-after-covid-19

TITLE: Sleep and Your School-Aged Child  (for Parents) - Nemours KidsHealth
SCORE: 275.2518005371094
URL: https://kidshealth.org/en/parents/sleep-school-age.html#catslee

TITLE: Kids and Sleep  (for Parents) - Nemours KidsHealth
SCORE: 268.62750244140625
URL: https://kidshealth.org/en/parents/sleep.html#catslee

